In [ ]:
#importing packages
!pip install pretrainedmodels
import pandas as pd
import os,time,random,cv2,shutil
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import pretrainedmodels
from tqdm import tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
!nvidia-smi

In [ ]:
#making data folders 
os.mkdir("./Train")
os.mkdir("./Valid")

val_csv = pd.read_csv("../input/annotations-v2/val_info_v2.csv")

src_path = "../input/ifood-2019-fgvc6/val_set/val_set/"
dst_path = "./Valid/"
for i in tqdm(range(len(val_csv))):
    img = val_csv["ImageID"][i]
    cls = val_csv["ClassName"][i]
    if(os.path.exists(dst_path+cls)):
        _=shutil.copy(src_path+img,dst_path+cls)
    else:
        os.mkdir(dst_path+cls)
        _=shutil.copy(src_path+img,dst_path+cls)

#Removing outliers
outliers = []
with open("./../input/annotations-v2/outliers.txt","r") as f:
    lines = f.readlines()
    for line in lines:
        outliers.append(line.split("\n")[0])
train_csv = pd.read_csv("../input/annotations-v2/train_info_v2.csv")

src_path = "../input/ifood-2019-fgvc6/train_set/train_set/"
dst_path = "./Train/"
for i in tqdm(range(len(train_csv))):
    img = train_csv["ImageID"][i]
    if img in outliers:
        continue
    cls = train_csv["ClassName"][i]
    if(os.path.exists(dst_path+cls)):
        _=shutil.copy(src_path+img,dst_path+cls)
    else:
        os.mkdir(dst_path+cls)
        _=shutil.copy(src_path+img,dst_path+cls)

In [ ]:
#functions for augmentation
def rotateImage(image):
    angle = np.random.randint(-25,25)
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

def brightness_more(image, factor=10):
    matrix = np.ones(image.shape, dtype = "uint8") * factor
    added = cv2.add(image, matrix)
    return added

def brightness_less(image, factor=10):
    matrix = np.ones(image.shape, dtype = "uint8") * factor
    subtracted = cv2.subtract(image, matrix)
    return subtracted

def hflip_image(image):
    return cv2.flip(image,1)

def contrast(image):
    #Converting image to LAB Color model
    lab= cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    #Splitting the LAB image to different channels
    l, a, b = cv2.split(lab)
    #Applying CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    #Merge the CLAHE enhanced L-channel with the a and b channel
    limg = cv2.merge((cl,a,b))
    #Converting image from LAB Color model to RGB model
    final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    return final

In [ ]:
#Augmenting the data
AUGMENTATIONS = [rotateImage,brightness_more,brightness_less,hflip_image,contrast]
new = 0
for classes in tqdm(os.listdir("./Train")):
    NUMBER_OF_IMAGES = len(os.listdir("./Train/"+classes))
    if (NUMBER_OF_IMAGES <= 100):
        #augmenting 200%
        REQ_IMAGES = int(NUMBER_OF_IMAGES + (NUMBER_OF_IMAGES*2.0))
            
    elif (NUMBER_OF_IMAGES < 300 and NUMBER_OF_IMAGES >= 200):
        #augmenting 100%
        REQ_IMAGES = int(NUMBER_OF_IMAGES + (NUMBER_OF_IMAGES*1.0))
            
    elif (NUMBER_OF_IMAGES < 350 and NUMBER_OF_IMAGES >= 300):
        #augmenting 50%
        REQ_IMAGES = int(NUMBER_OF_IMAGES + (NUMBER_OF_IMAGES*0.5))

    elif (NUMBER_OF_IMAGES < 400 and NUMBER_OF_IMAGES >= 350):
        #augmenting 30%
        REQ_IMAGES = int(NUMBER_OF_IMAGES + (NUMBER_OF_IMAGES*0.3))
            
    elif (NUMBER_OF_IMAGES < 450 and NUMBER_OF_IMAGES >= 400):
        #augmenting 10%
        REQ_IMAGES = int(NUMBER_OF_IMAGES + (NUMBER_OF_IMAGES*0.1))
            
    else:
        continue
        
    while(len(os.listdir("./Train/"+classes)) != REQ_IMAGES):
        NUMBER_OF_AUGMENTATIONS = np.random.randint(1,len(AUGMENTATIONS)+1)
        ORIGINAL_IMAGE = random.choice(os.listdir("./Train/"+classes))
        ORIG_IMG = cv2.imread("./Train/"+classes+"/"+ORIGINAL_IMAGE)
        for i in range(NUMBER_OF_AUGMENTATIONS):
            AUG = random.choice(AUGMENTATIONS)
            NEW_IMG = AUG(ORIG_IMG)
            ORIG_IMG = NEW_IMG
        cv2.imwrite("./Train/"+classes+"/"+ORIGINAL_IMAGE.split(".")[0]+"_"+str(new)+".jpg",NEW_IMG)
        new+=1